### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import os
import pandas as pd

from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, precision_score, accuracy_score, recall_score, roc_curve, auc
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.math import confusion_matrix
from myscripts import array_dict, get_metrics_df, find_combined_img
import SimpleITK as sitk

np.random.seed(24)

## Used Models

---

### T2 CNN Model

#### Setup

In [3]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)]

In [9]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "t2_tse_tra_192_0" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    if 'dcm' in each:
                        try:
                            image = sitk.ReadImage(image_c_dir+each)
                            image_arr = sitk.GetArrayFromImage(image)/255
                            control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_arr})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.')    

9480 pictures converted.


In [11]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)]

In [12]:
# list of dictionaries for all schizophrenic image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "t2_tse_tra_192_0" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    try:  
                        image = sitk.ReadImage(image_s_dir+each)
                        image_arr = sitk.GetArrayFromImage(image)/255
                        schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_arr})
                    except:
                        print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.')  

8040 pictures converted.


In [13]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

X shape: (17520, 1, 192, 192)
y shape: (17520,)


#### Train-test split

In [14]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, stratify=y)

In [15]:
# reshape to prep for cnn
X_train = np.array(X_train).reshape(13140, 192, 192, 1)

In [16]:
# checking shape
X_train.shape

(13140, 192, 192, 1)

In [17]:
# reshape to prep for cnn
X_test = np.array(X_test).reshape(4380, 192, 192, 1)

In [18]:
# checking shape
X_test.shape

(4380, 192, 192, 1)

#### Baseline Model

In [13]:

model = Sequential()

# conv & pool
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,2)))

# conv & pool
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 190, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 95, 95, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 93, 93, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 46, 46, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 270848)            0         
                                                                 
 dense (Dense)               (None, 64)                1

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
411/411 [==============================] - 624s 2s/step - loss: 0.6084 - accuracy: 0.6910 - val_loss: 0.3712 - val_accuracy: 0.7975
Epoch 2/10
411/411 [==============================] - 604s 1s/step - loss: 0.2702 - accuracy: 0.8555 - val_loss: 0.2368 - val_accuracy: 0.8683
Epoch 3/10
411/411 [==============================] - 616s 1s/step - loss: 0.1699 - accuracy: 0.9033 - val_loss: 0.2165 - val_accuracy: 0.8811
Epoch 4/10
411/411 [==============================] - 596s 1s/step - loss: 0.1497 - accuracy: 0.9145 - val_loss: 0.2827 - val_accuracy: 0.8612
Epoch 5/10
411/411 [==============================] - 609s 1s/step - loss: 0.1312 - accuracy: 0.9259 - val_loss: 0.2346 - val_accuracy: 0.8822
Epoch 6/10
411/411 [==============================] - 618s 2s/step - loss: 0.1142 - accuracy: 0.9322 - val_loss: 0.2366 - val_accuracy: 0.8900
Epoch 7/10
411/411 [==============================] - 599s 1s/step - loss: 0.1111 - accuracy: 0.9366 - val_loss: 0.2640 - val_accuracy: 0.8783

In [17]:
model.evaluate(X_test, y_test, batch_size=32, workers=-1)

137/137 [==============================] - 55s 402ms/step - loss: 0.2465 - accuracy: 0.8947


[0.24648414552211761, 0.894748866558075]

In [19]:
# saving first model
model.save('../models/model_first_t2.h5')

---

### Mprage CNN Model

#### Setup

In [ ]:
control_array_dict = array_dict('control', 'mprage', 'all', 'dcm')

In [4]:
schiz_array_dict = array_dict('schiz', 'mprage', 'all', 'dcm')

22656 pictures converted.


In [4]:
# setting up X and y
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
# X_arr = np.array(X, dtype=object)
# print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

y shape: (45504,)


#### Train test split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=24)

In [11]:
X_train = np.array(X_train).reshape(len(X_train), 256, 256, 1)

In [12]:
X_train.shape

(34128, 256, 256, 1)

In [13]:
X_test = np.array(X_test).reshape(len(X_test), 256, 256, 1)

In [14]:
X_test.shape

(11376, 256, 256, 1)

##### Saving

In [18]:
# saving X_train, y_train, X_test, y_test because computationally intensive
np.save('X_train', X_train)

np.save('X_test', X_test)

np.save('y_train', y_train)

np.save('y_test', y_test)

##### Loadings

In [5]:
X_train = np.load('X_train.npy')

In [6]:
X_test = np.load('X_test.npy')

In [7]:
y_train = np.load('y_train.npy')

In [8]:
y_test = np.load('y_test.npy')

#### Modeling

In [7]:
model = Sequential()

# conv & pool
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,2)))

# conv & pool
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 492032)            0         
                                                                 
 dense (Dense)               (None, 64)                3

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
1067/1067 [==============================] - 2870s 3s/step - loss: 0.5026 - accuracy: 0.7146 - val_loss: 0.3141 - val_accuracy: 0.8345
Epoch 2/10
1067/1067 [==============================] - 2792s 3s/step - loss: 0.2283 - accuracy: 0.8858 - val_loss: 0.1936 - val_accuracy: 0.9039
Epoch 3/10
1067/1067 [==============================] - 2757s 3s/step - loss: 0.1319 - accuracy: 0.9363 - val_loss: 0.1974 - val_accuracy: 0.9147
Epoch 4/10
1067/1067 [==============================] - 2742s 3s/step - loss: 0.1014 - accuracy: 0.9533 - val_loss: 0.1397 - val_accuracy: 0.9370
Epoch 5/10
1067/1067 [==============================] - 2736s 3s/step - loss: 0.0706 - accuracy: 0.9658 - val_loss: 0.1315 - val_accuracy: 0.9424
Epoch 6/10
1067/1067 [==============================] - 2721s 3s/step - loss: 0.0735 - accuracy: 0.9675 - val_loss: 0.1412 - val_accuracy: 0.9398
Epoch 7/10
1067/1067 [==============================] - 2748s 3s/step - loss: 0.0558 - accuracy: 0.9755 - val_loss: 0.1495 -

In [12]:
# saving full model
model.save('../models/model_full_mprage.h5')

## Unused Models

---

### Unknown Model 1

**Note**: Testing just to see what happens

#### Setup

In [2]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)]

In [3]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "t2_tse_tra_192_0" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    if "nii" in each:
                        try:
                            image = sitk.ReadImage(image_c_dir+each)
                            image_array = sitk.GetArrayFromImage(image)
                            image_flip = np.flip(image)
                            control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_flip})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.') 

79 pictures converted.


In [4]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)]

In [5]:
# list of dicitionarys for control image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "t2_tse_tra_192_0" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    if "nii" in each:
                        try:
                            image = sitk.ReadImage(image_s_dir+each)
                            image_array = sitk.GetArrayFromImage(image)
                            image_flip = np.flip(image)
                            schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_flip})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.') 

67 pictures converted.


In [6]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

X shape: (146, 4423680)
y shape: (146,)


#### Train test split

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, stratify=y)

In [8]:
# reshape to prep for cnn
X_train = np.array(X_train).reshape(109, 192, 192, 120)

In [9]:
# checking shape
X_train.shape

(109, 192, 192, 120)

In [10]:
# reshape to prep for cnn
X_test = np.array(X_test).reshape(37, 192, 192, 120)

In [11]:
# checking shape
X_test.shape

(37, 192, 192, 120)

#### Modeling

In [16]:
model = Sequential()

# conv & pool
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=X_train[0].shape))
model.add(AveragePooling2D(pool_size=(2,2)))

# conv & pool
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 190, 190, 64)      69184     
                                                                 
 average_pooling2d (AverageP  (None, 95, 95, 64)       0         
 ooling2D)                                                       
                                                                 
 conv2d_3 (Conv2D)           (None, 93, 93, 128)       73856     
                                                                 
 average_pooling2d_1 (Averag  (None, 46, 46, 128)      0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 270848)            0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
4/4 [==============================] - 14s 3s/step - loss: 14384.4561 - accuracy: 0.4862 - val_loss: 818.6246 - val_accuracy: 0.5405
Epoch 2/10
4/4 [==============================] - 13s 3s/step - loss: 677.7670 - accuracy: 0.4862 - val_loss: 194.7544 - val_accuracy: 0.4865
Epoch 3/10
4/4 [==============================] - 13s 3s/step - loss: 240.7955 - accuracy: 0.4312 - val_loss: 13.3596 - val_accuracy: 0.5946
Epoch 4/10
4/4 [==============================] - 15s 4s/step - loss: 87.1284 - accuracy: 0.5321 - val_loss: 11.3811 - val_accuracy: 0.5405
Epoch 5/10
4/4 [==============================] - 18s 4s/step - loss: 14.8301 - accuracy: 0.5413 - val_loss: 12.5962 - val_accuracy: 0.4595
Epoch 6/10
4/4 [==============================] - 18s 4s/step - loss: 12.0230 - accuracy: 0.5138 - val_loss: 12.0389 - val_accuracy: 0.5405
Epoch 7/10
4/4 [==============================] - 17s 4s/step - loss: 3.9540 - accuracy: 0.6055 - val_loss: 4.6829 - val_accuracy: 0.5405
Epoch 8/10
4/4 

---

### Gate Model

In [2]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)]

In [3]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "GATE_V01_R" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    try:
                        image = sitk.ReadImage(image_c_dir+each)
                        image_arr = sitk.GetArrayFromImage(image)/255
                        control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_arr})
                    except:
                        print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.')    

2016 pictures converted.


In [4]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)][:7]

In [5]:
# list of dictionaries for all schizophrenic image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "GATE_V01_R" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    try:  
                        image = sitk.ReadImage(image_s_dir+each)
                        image_arr = sitk.GetArrayFromImage(image)/255
                        schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_arr})
                    except:
                        print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.')  

2352 pictures converted.


In [6]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

X shape: (4368, 1, 384, 384)
y shape: (4368,)


#### Train-test split

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, stratify=y)

In [8]:
# reshape to prep for cnn
X_train = np.array(X_train).reshape(3276, 384, 384, 1)

In [9]:
# checking shape
X_train.shape

(3276, 384, 384, 1)

In [10]:
# reshape to prep for cnn
X_test = np.array(X_test).reshape(1092, 384, 384, 1)

In [11]:
# checking shape
X_test.shape

(1092, 384, 384, 1)

#### Baseline Model

In [12]:
model = Sequential()

# conv & pool
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,2)))

# conv & pool
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 382, 382, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 191, 191, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 189, 189, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 94, 94, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1131008)           0         
                                                                 
 dense (Dense)               (None, 64)                7

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
103/103 [==============================] - 687s 6s/step - loss: 0.7001 - accuracy: 0.9484 - val_loss: 4.9919e-06 - val_accuracy: 1.0000
Epoch 2/10
103/103 [==============================] - 640s 6s/step - loss: 3.8963e-06 - accuracy: 1.0000 - val_loss: 3.3476e-06 - val_accuracy: 1.0000
Epoch 3/10
103/103 [==============================] - 757s 7s/step - loss: 2.7207e-06 - accuracy: 1.0000 - val_loss: 2.4626e-06 - val_accuracy: 1.0000
Epoch 4/10
103/103 [==============================] - 753s 7s/step - loss: 2.0261e-06 - accuracy: 1.0000 - val_loss: 1.8947e-06 - val_accuracy: 1.0000
Epoch 5/10
103/103 [==============================] - 751s 7s/step - loss: 1.5674e-06 - accuracy: 1.0000 - val_loss: 1.4974e-06 - val_accuracy: 1.0000
Epoch 6/10
103/103 [==============================] - 710s 7s/step - loss: 1.2488e-06 - accuracy: 1.0000 - val_loss: 1.2118e-06 - val_accuracy: 1.0000
Epoch 7/10
103/103 [==============================] - 668s 6s/step - loss: 1.0180e-06 - accuracy: 

---

### Dti Model

In [2]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)][:10]

In [3]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "dti" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    if 'nii' not in each:
                        # print(each)
                        try:
                            image = sitk.ReadImage(image_c_dir+each)
                            image_arr = sitk.GetArrayFromImage(image)/255
                            control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_arr})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.')    

700 pictures converted.


In [5]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)][:10]

In [6]:
# list of dictionaries for all schizophrenic image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "dti" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    if 'nii' not in each:
                        # print(each)
                        try:  
                            image = sitk.ReadImage(image_s_dir+each)
                            image_arr = sitk.GetArrayFromImage(image)/255
                            schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_arr})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.')  

675 pictures converted.


In [12]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

MemoryError: Unable to allocate 19.8 GiB for an array with shape (2005, 1, 1152, 1152) and data type object

* Images were the largest at 1152 x 1152
* Scan abandoned for now

---

### RST Model

In [2]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)][:12]

In [3]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "RST" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    if 'nii' not in each:
                        # print(each)
                        try:
                            image = sitk.ReadImage(image_c_dir+each)
                            image_arr = sitk.GetArrayFromImage(image)/255
                            control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_arr})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.')    

1650 pictures converted.


In [4]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)][:10]

In [5]:
# list of dictionaries for all schizophrenic image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "RST" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    if 'nii' not in each:
                        # print(each)
                        try:  
                            image = sitk.ReadImage(image_s_dir+each)
                            image_arr = sitk.GetArrayFromImage(image)/255
                            schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_arr})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.')  

1650 pictures converted.


In [6]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

X shape: (3300, 1, 384, 384)
y shape: (3300,)


#### Train-test split

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, stratify=y)

In [8]:
# reshape to prep for cnn
X_train = np.array(X_train).reshape(2475, 384, 384, 1)

In [9]:
# checking shape
X_train.shape

(2475, 384, 384, 1)

In [10]:
# reshape to prep for cnn
X_test = np.array(X_test).reshape(825, 384, 384, 1)

In [11]:
# checking shape
X_test.shape

(825, 384, 384, 1)

#### Baseline Model

In [12]:
model = Sequential()

# conv & pool
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,2)))

# conv & pool
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 382, 382, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 191, 191, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 189, 189, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 94, 94, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1131008)           0         
                                                                 
 dense (Dense)               (None, 64)                7

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
78/78 [==============================] - 492s 6s/step - loss: 0.4000 - accuracy: 0.9612 - val_loss: 1.8154e-07 - val_accuracy: 1.0000
Epoch 2/10
78/78 [==============================] - 470s 6s/step - loss: 1.5466e-07 - accuracy: 1.0000 - val_loss: 9.0894e-08 - val_accuracy: 1.0000
Epoch 3/10
78/78 [==============================] - 468s 6s/step - loss: 8.7591e-08 - accuracy: 1.0000 - val_loss: 7.3799e-08 - val_accuracy: 1.0000
Epoch 4/10
78/78 [==============================] - 462s 6s/step - loss: 7.1835e-08 - accuracy: 1.0000 - val_loss: 6.3943e-08 - val_accuracy: 1.0000
Epoch 5/10
78/78 [==============================] - 464s 6s/step - loss: 6.2163e-08 - accuracy: 1.0000 - val_loss: 5.6111e-08 - val_accuracy: 1.0000
Epoch 6/10
78/78 [==============================] - 463s 6s/step - loss: 5.4743e-08 - accuracy: 1.0000 - val_loss: 4.9667e-08 - val_accuracy: 1.0000
Epoch 7/10
19/78 [======>.......................] - ETA: 6:23 - loss: 5.2769e-08 - accuracy: 1.0000

* Model crashed before saving and finishing however it looks promising 

---

### RST Model (3D)

In [47]:
# check image and shape
image = sitk.ReadImage('../brain_images/schizophrenic/cobre_07325/A00000368/Session_1/RSTpre_V01_R01_0011/RSTpre_V01_R01+CM.nii') 
img_arr = sitk.GetArrayFromImage(image)
print(img_arr.shape)

(150, 33, 64, 64)


#### Setup

In [11]:
# setting initial control scan path
control_dir = '../brain_images/no_known_disorder/cobre_07325/'

# setting total control patients
patients_c = [each for each in os.listdir(control_dir)][:12]

In [12]:
# list of dicitionarys for control image arrays
control_arrays_dict = []
for patient in patients_c:
    for session in os.listdir(control_dir+patient):
        for scan in os.listdir(control_dir+patient+'/'+session): 
            if "RSTpre_V01_" in scan:
                image_c_dir = control_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_c_dir):
                    if "CM" in each:
                        try:
                            image = sitk.ReadImage(image_c_dir+each)
                            image_array = sitk.GetArrayFromImage(image)/255
                            control_arrays_dict.append({'path': str(image_c_dir+each), 'scan': image_array})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(control_arrays_dict)} pictures converted.') 

10 pictures converted.


In [13]:
# setting initial schizophrenic image path
schiz_dir = '../brain_images/schizophrenic/cobre_07325/'

# setting total schizophrenic patients
patients_s = [each for each in os.listdir(schiz_dir)][:10]

In [14]:
# list of dicitionarys for control image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "RSTpre_V01_" in scan:
                image_s_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                for each in os.listdir(image_s_dir):
                    if "CM" in each:
                        try:
                            image = sitk.ReadImage(image_s_dir+each)
                            image_array = sitk.GetArrayFromImage(image)/255
                            schiz_arrays_dict.append({'path': str(image_s_dir+each), 'scan': image_array})
                        except:
                            print(f'Error for file: {each}')
print(f'{len(schiz_arrays_dict)} pictures converted.') 

10 pictures converted.


In [15]:
# combining control and schizophrenic scans
X = [each['scan'] for each in control_arrays_dict] + [each['scan'] for each in schiz_arrays_dict]

# convert to array and checking for shape of arrays
X_arr = np.array(X, dtype=object)
print(f'X shape: {X_arr.shape}')

# setting control scans to 0 and schizophrenic scans to 1
y = [0] * len(control_arrays_dict) + [1] * len(schiz_arrays_dict)

# convert to array and check shape
y = np.array(y)
print(f'y shape: {y.shape}')

X shape: (20, 150, 33, 64, 64)
y shape: (20,)


#### Train test split

In [16]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, stratify=y)

In [21]:
# reshape to prep for cnn
X_train = np.array(X_train).reshape(15, 64, 64, 150, 33)

In [22]:
# checking shape
X_train.shape

(15, 64, 64, 150, 33)

In [23]:
# reshape to prep for cnn
X_test = np.array(X_test).reshape(5, 64, 64, 150, 33)

In [24]:
# checking shape
X_test.shape

(5, 64, 64, 150, 33)

#### Modeling (3d)

In [27]:
model = Sequential()

# conv & pool
model.add(Conv3D(filters=64, kernel_size=(3,3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling3D(pool_size=(2,2,2)))

# conv & pool
model.add(Conv3D(filters=128, kernel_size=(3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))

# flatten & finish
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid')) # binary classification 

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 62, 62, 148, 64)   57088     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 31, 31, 74, 64)   0         
 3D)                                                             
                                                                 
 conv3d_4 (Conv3D)           (None, 29, 29, 72, 128)   221312    
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 14, 14, 36, 128)  0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 903168)            0         
                                                                 
 dense (Dense)               (None, 64)               

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
1/1 [==============================] - 103s 103s/step - loss: 3.6765 - accuracy: 0.6667 - val_loss: 42571.7773 - val_accuracy: 0.6000
Epoch 2/10
1/1 [==============================] - 80s 80s/step - loss: 63191.0391 - accuracy: 0.4667 - val_loss: 3308.0867 - val_accuracy: 0.6000
Epoch 3/10
1/1 [==============================] - 87s 87s/step - loss: 4927.9551 - accuracy: 0.4667 - val_loss: 2815.7751 - val_accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 76s 76s/step - loss: 2341.0791 - accuracy: 0.5333 - val_loss: 370.0012 - val_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 81s 81s/step - loss: 307.5126 - accuracy: 0.5333 - val_loss: 1499.0067 - val_accuracy: 0.6000
Epoch 6/10
1/1 [==============================] - 74s 74s/step - loss: 1985.2516 - accuracy: 0.4667 - val_loss: 735.1409 - val_accuracy: 0.6000
Epoch 7/10
1/1 [==============================] - 80s 80s/step - loss: 929.3458 - accuracy: 0.4667 - val_loss: 116.1030 - val_accura

* Sample size too small to make inferences
* Computationally intensive for setting up arrays not for model

---

### Unfinished setup for custom 3D nii files

In [108]:
image = nib.load('../brain_images/no_known_disorder/cobre_07325/A00000300/Session_2/t2_tse_tra_192_0005/t2_tse_tra_192_0005_t2_tse_tra_192_0_5.nii')
image.shape

(192, 192, 120)

In [97]:
image_array = np.asarray(image)

In [98]:
image_array = np.rot90(image_array)

In [131]:
image = sitk.ReadImage('../brain_images/no_known_disorder/cobre_07325/A00000300/Session_2/t2_tse_tra_192_0005/t2_tse_tra_192_0005_t2_tse_tra_192_0_5.nii') 
img_arr = sitk.GetArrayFromImage(image)
img_arr = np.flip(img_arr)
print(img_arr.shape)

(120, 192, 192)


In [111]:
image_rot_arr.shape

(192, 192, 120)

In [17]:
from nipype.interfaces.dcm2nii import Dcm2niix

In [18]:
# list of dictionaries for all schizophrenic image arrays
schiz_arrays_dict = []
for patient in patients_s:
    for session in os.listdir(schiz_dir+patient):
        for scan in os.listdir(schiz_dir+patient+'/'+session): 
            if "t2_tse_tra_192_0" in scan:
                # print(scan)
                try:
                    converter = Dcm2niix()
                    converter.inputs.source_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                    converter.inputs.compress = 'n'
                    converter.inputs.output_dir = schiz_dir+patient+'/'+session+'/'+scan+'/'
                    converter.run()
                except:
                    print(f'Error for file: {each}')

220226-02:22:40,231 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,254 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,274 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,298 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,324 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,362 nipype.interface INFO:
	 [WinError 10038] An operation was attempted on something that is not a socket
Error for file: SerieMR-0004-0120-1.dcm
220226-02:22:40,407 ni